In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
films = pd.read_csv("netflix-rotten-tomatoes-metacritic-imdb.csv")
films

,Title,Genre,Tags,Languages,Series or Movie,Hidden Gem Score,Country Availability,Runtime,Director,Writer,Actors,View Rating,IMDb Score,Rotten Tomatoes Score,Metacritic Score,Awards Received,Awards Nominated For,Boxoffice,Release Date,Netflix Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer,Trailer Site
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,4.3,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,"Kåre Hedebrant, Per Ragnar, Lina Leandersson, ...",R,7.9,98.0,82.0,74.0,57.0,"$2,122,065",12 Dec 2008,2021-03-04,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,NaN,NaN
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,7.0,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,"Paddy Considine, Cleo, Beanie Feldstein, Dónal...",R,5.8,79.0,69.0,1.0,NaN,"$70,632",08 May 2020,2021-03-04,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ,YouTube
2,Centigrade,"Drama, Thriller",Thrillers,English,Movie,6.4,Canada,1-2 hour,Brendan Walsh,"Brendan Walsh, Daley Nixon","Genesis Rodriguez, Vincent Piazza",Unrated,4.3,NaN,46.0,NaN,NaN,"$16,263",28 Aug 2020,2021-03-04,NaN,https://www.netflix.com/watch/81305978,https://www.imdb.com/title/tt8945942,"Trapped in a frozen car during a blizzard, a p...",1720.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BODM2MD...,https://www.youtube.com/watch?v=0RvV7TNUlkQ,YouTube
3,ANNE+,Drama,"TV Dramas,Romantic TV Dramas,Dutch TV Shows",Turkish,Series,7.7,"Belgium,Netherlands",< 30 minutes,NaN,NaN,"Vahide Perçin, Gonca Vuslateri, Cansu Dere, Be...",NaN,6.5,NaN,NaN,1.0,NaN,NaN,01 Oct 2016,2021-03-04,NaN,https://www.netflix.com/watch/81336456,https://www.imdb.com/title/tt6132758,"Upon moving into a new place, a 20-something r...",1147.0,https://occ-0-1489-1490.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BNWRkMz...,NaN,NaN
4,Moxie,"Animation, Short, Drama","Social Issue Dramas,Teen Movies,Dramas,Comedie...",English,Movie,8.1,"Lithuania,Poland,France,Iceland,Italy,Spain,Gr...",1-2 hour,Stephen Irwin,NaN,Ragga Gudrun,NaN,6.3,NaN,NaN,NaN,4.0,NaN,22 Sep 2011,2021-03-04,NaN,https://www.netflix.com/watch/81078393,https://www.imdb.com/title/tt2023611,Inspired by her moms rebellious past and a con...,63.0,https://occ-0-4039-1500.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BODYyNW...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15475,K-POP Extreme Survival,NaN,"TV Dramas,TV Programmes,TV Comedies,Romantic T...",NaN,Series,NaN,"South Korea,Argentina,United Kingdom,Australia...",< 30 minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-04-14,NaN,https://www.netflix.com/watch/80031595,NaN,Seung Yeon decides to chase her dream of becom...,NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN
15476,DreamWorks Shreks Swamp Stories,NaN,"Animal Tales,Family Comedies,Family Adventures...",NaN,Series,NaN,"Russia,Hong Kong,Hungary,Australia,South Korea...",< 30 minutes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-04-14,NaN,https://www.netflix.com/watch/70242630,NaN,"Shrek celebrates Halloween, Puss in Boots is c...",NaN,https://occ-0-2851-38.1.nflxso.net/dnm/api/v6/...,NaN,NaN,NaN
15477,DreamWorks Happy Holidays from Madagascar,"Animation, Comedy, Family","TV Comedies,Kids TV,Animal Tales,TV Carto

In [9]:
def query_processor(query):
    query = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", query)
    query = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", query)
    query = re.sub("'", "", query)
    query = re.sub(",", "", query)
    tokens = re.split(r"\s+",query)
    tokens = [t.lower() for t in tokens]
    # Stemming with the SnowballStemmer for more efficiency
    s_stemmer = SnowballStemmer("english")
    stemedList = []
    for word in tokens:
        stemedList.append(s_stemmer.stem(word))
    tokens = stemedList
    # Lemmatising the tokens
    wordnet_lemmatiser = WordNetLemmatizer()
    lemmaList = []
    for word in stemedList:
        lemmaList.append(wordnet_lemmatiser.lemmatize(word))
    tokens = lemmaList
    # Finally remove stopwords
    stops = set(stopwords.words("english"))
    tokens = [word for word in tokens if not word in stops]
    return tokens

def dataset_processor(dataset):
    dataset = pd.read_csv("netflix-rotten-tomatoes-metacritic-imdb.csv")
    dataset = dataset.drop(columns = ["Trailer Site", "TMDb Trailer", "Poster", "Image", "IMDb Votes", "Netflix Link", "Production House", "Netflix Release Date", "Boxoffice", "Awards Received", "Awards Nominated For", "IMDb Link", "Runtime", "Country Availability", "View Rating"])
    dataset["Average Score"] = dataset.apply(Average, axis = 1)
    dataset = dataset.drop(columns = ["Hidden Gem Score", "IMDb Score", "Rotten Tomatoes Score", "Metacritic Score"])
    dataset = dataset.astype({"Title":"str", "Genre":"str", "Tags":"str",
                     "Languages":"str", "Series or Movie":"category",
                     "Director":"str", "Writer":"str", "Actors":"str",
                     "Summary":"str", "Average Score":"float"})
    # So need to query processor the summary, genre list and tag list.
    for i in ["Summary", "Genre", "Tags"]:
      dataset[i] = dataset.loc[:, i].apply(query_processor)

    # Need to turn the director, writer, languages, and actors into a list.
    for i in ["Director", "Languages", "Writer", "Actors"]:
      dataset[i] = dataset.loc[:,i].apply(Split)

    # Change the release data to just the year
    dataset["Release Date"] = dataset["Release Date"].apply(Year)
    return dataset

def Split(row):
  tokens = re.split(",", row)
  return tokens

def Year(row):
  tokens = re.split(" ", str(row))
  if tokens[0] == "nan":
    return "NaN"
  else:
    return tokens[2]

def Average(row):
  scores = []
  scorers = ["Hidden Gem Score", "IMDb Score", "Rotten Tomatoes Score", "Metacritic Score"]
  potential = [10,10,100,100]
  for i in range(len(scorers)):
    if (np.isnan(row.loc[scorers[i]])) == False:
      scores.append(row.loc[scorers[i]]/potential[i])
  if len(scores) > 0:
    return sum(scores)/len(scores)
  else:
    return "NaN"

In [7]:
films = pd.read_csv("netflix-rotten-tomatoes-metacritic-imdb.csv")
films = dataset_processor(films)
films

,Title,Genre,Tags,Languages,Series or Movie,Director,Writer,Actors,Release Date,Summary,Average Score
0,Lets Fight Ghost,"[crime, drama, fantasi, horror, romanc]","[comedi, programm, romant, tv, comedi, horror,...","[Swedish, Spanish]",Series,[Tomas Alfredson],[John Ajvide Lindqvist],"[Kåre Hedebrant, Per Ragnar, Lina Leandersso...",2008,"[med, student, supernatur, gift, tri, cash, ab...",0.755
1,HOW TO BUILD A GIRL,[comedi],"[drama, comedi, film, base, book, british]",[English],Movie,[Coky Giedroyc],[Caitlin Moran],"[Paddy Considine, Cleo, Beanie Feldstein, D...",2020,"[nerdi, johanna, move, london, thing, get, han...",0.690
2,Centigrade,"[drama, thriller]",[thriller],[English],Movie,[Brendan Walsh],"[Brendan Walsh, Daley Nixon]","[Genesis Rodriguez, Vincent Piazza]",2020,"[trap, frozen, car, dure, blizzard, pregnant, ...",0.510
3,ANNE+,[drama],"[tv, drama, romant, tv, drama, dutch, tv, show]",[Turkish],Series,[nan],[nan],"[Vahide Perçin, Gonca Vuslateri, Cansu Dere,...",2016,"[upon, move, new, place, 20-someth, run, forme...",0.710
4,Moxie,"[anim, short, drama]","[social, issu, drama, teen, movi, drama, comed...",[English],Movie,[Stephen Irwin],[nan],[Ragga Gudrun],2011,"[inspir, mom, rebelli, past, confid, new, frie...",0.720
...,...,...,...,...,...,...,...,...,...,...,...
15475,K-POP Extreme Survival,[nan],"[tv, drama, tv, programm, tv, comedi, romant, ...",[nan],Series,[nan],[nan],[nan],NaN,"[seung, yeon, decid, chase, dream, becom, k-po...",NaN
15476,DreamWorks Shreks Swamp Stories,[nan],"[anim, tale, famili, comedi, famili, adventur,...",[nan],Series,[nan],[nan],[nan],NaN,"[shrek, celebr, halloween, pus, boot, captur, ...",NaN
15477,DreamWorks Happy Holidays from Madagascar,"[anim, comedi, famili]","[tv, comedi, kid, tv, anim, tale, tv, cartoon,...",[English],Series,[nan],[nan],[Jung Hyun Kim],NaN,"[madagascar, goe, wild, holiday, spirit, set, ...",0.760
15478,DreamWorks Holiday Classics,"[anim, comedi, famili]","[tv, comedi, kid, tv, tv, cartoon, tv, program...",[English],Series,[nan],[nan],[nan],NaN,"[join, dreamwork, friend, four, holiday, speci...",0.730


In [ ]:
query = "I want to watch something crime related"
query = query_processor(query)
print(query)

['want', 'watch', 'someth', 'crime', 'relat']
